In [1]:
import os
os.chdir("/Users/victorcarre/Code/Base de donnée biostimulants/bio4safe-scraper")

import warnings
warnings.filterwarnings("ignore", message="NotOpenSSLWarning")

import requests
import pandas as pd
import pubchempy as pcp

data = pd.read_csv("data/biostimulants.csv", sep=";", encoding="ISO-8859-1")

print(data.head())

biostimulants = data['biostimulant'].unique()

print("Types uniques : ", data['type'].unique())
print("Effets uniques : ", data['effect'].unique())

/Users/victorcarre/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


      biostimulant                    type                              effect
0              Amf       Mycorrhizal Fungi  Decreased drought stress tolerance
1      Bioatlantis         Seaweed Extract  Decreased drought stress tolerance
2          Biomega                Bacteria  Decreased drought stress tolerance
3           Kelpak         Seaweed Extract  Decreased drought stress tolerance
4  Naturvital-Plus  Humic And Fluvic Acids  Decreased drought stress tolerance
Types uniques :  ['Mycorrhizal Fungi' 'Seaweed Extract' 'Bacteria' 'Humic And Fluvic Acids'
 'Anorganic Elements Si, Phosphite' 'Mix Of Different Types' 'Other'
 'Protein Hydrolysate' 'Plant Extract' 'Chitosan & Biopolymers'
 'Data Not Available']
Effets uniques :  ['Decreased drought stress tolerance' 'Decreased growth'
 'Decreased heat stress tolerance' 'Decreased nitrogen n use efficiency'
 'Decreased nutrient availability in soil or substrate' 'Decreased other'
 'Decreased phosphorous p use efficiency'
 'Decreased po

In [2]:
# Création des colonnes vides
data["SMILES"] = ""
data["Molecular Weight"] = ""
data["Brand"] = ""  # On ajoutera les marques plus tard

# Remplir les codes SMILES et les poids moléculaires
for index, row in data.iterrows():
    name = row["biostimulant"]
    try:
        # Rechercher la première correspondance sur PubChem
        compound = pcp.get_compounds(name, 'name')
        if compound:
            # SMILES
            smiles = compound[0].canonical_smiles
            if Chem.MolFromSmiles(smiles):
                data.at[index, "SMILES"] = smiles
                
                # Poids moléculaire
                mol_weight = compound[0].molecular_weight
                if mol_weight:
                    data.at[index, "Molecular Weight"] = mol_weight
                
                print(f"✅ Données trouvées pour {name}: SMILES = {smiles}, MW = {mol_weight}")
            else:
                print(f"⚠️ SMILES invalide pour {name}")
        else:
            print(f"⚠️ Aucun composé trouvé pour {name}")
    except Exception as e:
        print(f"⚠️ Erreur avec {name}: {e}")

# Sauvegarder une seule fois
data.to_csv("data/biostimulants_with_SMILES_MW.csv", sep=";", index=False, encoding="ISO-8859-1")
print("\n✅ Fichier sauvegardé avec les SMILES et les poids moléculaires")


⚠️ Aucun composé trouvé pour Amf
⚠️ Aucun composé trouvé pour Bioatlantis
⚠️ Erreur avec Biomega: name 'Chem' is not defined
⚠️ Aucun composé trouvé pour Kelpak
⚠️ Aucun composé trouvé pour Naturvital-Plus
⚠️ Aucun composé trouvé pour Phylgreen
⚠️ Aucun composé trouvé pour Trianum-P
⚠️ Aucun composé trouvé pour Basfoliar Si
⚠️ Aucun composé trouvé pour Fitostim
⚠️ Aucun composé trouvé pour Instant Td + Impuls Td
⚠️ Aucun composé trouvé pour Phylgreen
⚠️ Aucun composé trouvé pour Previsan S
⚠️ Aucun composé trouvé pour Siapton
⚠️ Aucun composé trouvé pour Calleaf Aminovital
⚠️ Aucun composé trouvé pour Kelpak
⚠️ Aucun composé trouvé pour Maximus Amino Protect
⚠️ Aucun composé trouvé pour Nutricin
⚠️ Aucun composé trouvé pour Tnc Bactorrs13
⚠️ Aucun composé trouvé pour Trianum-P
⚠️ Aucun composé trouvé pour True Algae Max (Tam)
⚠️ Aucun composé trouvé pour Vitact R
⚠️ Aucun composé trouvé pour Golf Algin
⚠️ Aucun composé trouvé pour Maxicrop
⚠️ Aucun composé trouvé pour Progreen
⚠️ Aucun

In [2]:
# Exemple de fonction de scraping pour récupérer les marques
def get_brand_from_website(biostimulant_name):
    # Remplacer l'URL par celle du site réel où tu obtiens les marques
    url = f"https://www.example.com/search?q={biostimulant_name}"
    
    response = requests.get(url)
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Exemple de recherche dans le HTML pour trouver le nom de la marque (modifier selon la structure réelle de la page)
        brand_tag = soup.find('span', class_='brand')  # Cela dépend du site, il faut ajuster les sélecteurs
        
        if brand_tag:
            return brand_tag.text.strip()  # Retourne le nom de la marque
        else:
            return 'Marque inconnue'  # Retourne une valeur par défaut si la marque n'est pas trouvée
    else:
        return 'Marque inconnue'  # En cas d'erreur HTTP

# Ajouter la colonne "marque" en fonction du nom du biostimulant
data['marque'] = data['biostimulant'].apply(lambda x: get_brand_from_website(x))
# Ajouter la colonne "marque" en fonction du nom du biostimulant
data['marque'] = data['biostimulant'].apply(lambda x: biostimulant_to_brand.get(x, 'Inconnue'))  # 'Inconnue' si non trouvé

# Poids moléculaire depuis PubChem
def get_molecular_weight(compound_name):
    url = f"https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/{compound_name}/property/MolecularWeight/json"
    response = requests.get(url)
    if response.status_code == 200:
        try:
            mw = response.json()['PropertyTable']['Properties'][0]['MolecularWeight']
            return mw
        except KeyError:
            return "Non trouvé"
    else:
        return ""

In [3]:
# Récupérer les poids moléculaires (Attendre la complétion)
weights = {}
for b in biostimulants:
    weights[b] = get_molecular_weight(b)
print("Complété")

Complété


In [4]:
# Affichage des résultats
weights_df = pd.DataFrame(list(weights.items()), columns=['biostimulant', 'poids_moléculaire'])
print(weights_df)

        biostimulant poids_moléculaire
0                Amf                  
1        Bioatlantis                  
2            Biomega             305.4
3             Kelpak                  
4    Naturvital-Plus                  
..               ...               ...
142        Leafamine                  
143          Expando                  
144    Naturamin Wsp                  
145           Proact                  
146         Stimplex                  

[147 rows x 2 columns]


In [5]:
# Sauvegarder les résultats dans un CSV
data['poids_moleculaire'] = data['biostimulant'].apply(lambda x: get_molecular_weight(x))

data.to_csv("data/biostimulants_with_weights.csv", index=False)
print("Fichier sauvegardé")

Fichier sauvegardé


In [3]:
import pubchempy as pcp

# Test avec une molécule simple
test = pcp.get_compounds("glucose", "name")
print(test)

[Compound(5793)]


In [ ]:
import subprocess
from datetime import datetime

def run_command(command):
    subprocess.run(command, shell=True, check=True)

# Naviguer dans le répertoire
run_command("cd /chemin/vers/ton/projet")

# Ajouter les fichiers
run_command("git add .")

# Commit automatique avec la date et l'heure
commit_message = f"Auto commit {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}"
run_command(f"git commit -m \"{commit_message}\"")

# Pousser vers GitHub
run_command("git push origin main")
